In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

<IPython.core.display.Javascript object>

In [3]:
SETUP = False

<IPython.core.display.Javascript object>

In [4]:
if SETUP:
    !pip install -q -U toai
    !pip install -q -U nb_black
    !pip install -q -U tensorflow-datasets
    !pip install -q -U --no-deps tensorflow-addons
    !pip install -q -U tensorflow_hub
    print(__import__("toai").__version__)
    print(__import__("tensorflow").__version__)

<IPython.core.display.Javascript object>

In [5]:
print(__import__("toai").__version__)

0.3.7


<IPython.core.display.Javascript object>

In [6]:
from toai.imports import *
from toai.utils import save_file, load_file
from toai.data.utils import split_df
from toai.models import save_keras_model, load_keras_model
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


<IPython.core.display.Javascript object>

In [7]:
DATA_DIR = Path("data/wine-reviews")
TEMP_DIR = Path("temp/wine-reviews")

<IPython.core.display.Javascript object>

In [8]:
BATCH_SIZE = 512
AUTOTUNE = tf.data.experimental.AUTOTUNE

<IPython.core.display.Javascript object>

In [9]:
all_data = pd.read_csv(
    DATA_DIR / "winemag-data-130k-v2.csv", index_col=0, low_memory=False
)

<IPython.core.display.Javascript object>

In [10]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


<IPython.core.display.Javascript object>

In [11]:
all_data.describe(include="all")

country                                        description designation  \
count   129908                                             129971       92506   
unique      43                                             119955       37979   
top         US  Ripe plum, game, truffle, leather and menthol ...     Reserve   
freq     54504                                                  3        2009   
mean       NaN                                                NaN         NaN   
std        NaN                                                NaN         NaN   
min        NaN                                                NaN         NaN   
25%        NaN                                                NaN         NaN   
50%        NaN                                                NaN         NaN   
75%        NaN                                                NaN         NaN   
max        NaN                                                NaN         NaN   

               points          price    province     region_1       region_2  \
count   129971.000000  120975.000000      129908       108724          50511   
unique            NaN            NaN         425         1229             17   
top               NaN            NaN  California  Napa Valley  Central Coast   
freq              NaN            NaN       36247         4480          11065   
mean        88.447138      35.363389         NaN          NaN            NaN   
std          3.039730      41.022218         NaN          NaN            NaN   
min         80.000000       4.000000         NaN          NaN            NaN   
25%         86.000000      17.000000         NaN          NaN            NaN   
50%         88.000000      25.000000         NaN          NaN            NaN   
75%         91.000000      42.000000         NaN          NaN            NaN   
max        100.000000    3300.000000         NaN          NaN            NaN   

       taster_name taster_twitter_handle  \
count       103727                 98758   
unique          19                    15   
top     Roger Voss            @vossroger   
freq         25514                 25514   
mean           NaN                   NaN   
std            NaN                   NaN   
min            NaN                   NaN   
25%            NaN                   NaN   
50%            NaN                   NaN   
75%            NaN                   NaN   
max            NaN                   NaN   

                                                    title     variety  \
count                                              129971      129970   
unique                                             118840         707   
top     Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...  Pinot Noir   
freq                                                   11       13272   
mean                                                  NaN         NaN   
std                                                   NaN         NaN   
min                                                   NaN         NaN   
25%                                                   NaN         NaN   
50%                                                   NaN         NaN   
75%                                                   NaN         NaN   
max                                                   NaN         NaN   

                    winery  
count               129971  
unique               16757  
top     Wines & Winemakers  
freq                   222  
mean                   NaN  
std                    NaN  
min                    NaN  
25%                    NaN  
50%                    NaN  
75%                    NaN  
max                    NaN

<IPython.core.display.Javascript object>

In [12]:
all_data.head(10).T

0  \
country                                                            Italy   
description            Aromas include tropical fruit, broom, brimston...   
designation                                                 Vulkà Bianco   
points                                                                87   
price                                                                NaN   
province                                               Sicily & Sardinia   
region_1                                                            Etna   
region_2                                                             NaN   
taster_name                                                Kerin O’Keefe   
taster_twitter_handle                                       @kerinokeefe   
title                                  Nicosia 2013 Vulkà Bianco  (Etna)   
variety                                                      White Blend   
winery                                                           Nicosia   

                                                                       1  \
country                                                         Portugal   
description            This is ripe and fruity, a wine that is smooth...   
designation                                                     Avidagos   
points                                                                87   
price                                                                 15   
province                                                           Douro   
region_1                                                             NaN   
region_2                                                             NaN   
taster_name                                                   Roger Voss   
taster_twitter_handle                                         @vossroger   
title                      Quinta dos Avidagos 2011 Avidagos Red (Douro)   
variety                                                   Portuguese Red   
winery                                               Quinta dos Avidagos   

                                                                       2  \
country                                                               US   
description            Tart and snappy, the flavors of lime flesh and...   
designation                                                          NaN   
points                                                                87   
price                                                                 14   
province                                                          Oregon   
region_1                                               Willamette Valley   
region_2                                               Willamette Valley   
taster_name                                                 Paul Gregutt   
taster_twitter_handle                                        @paulgwine    
title                      Rainstorm 2013 Pinot Gris (Willamette Valley)   
variety                                                       Pinot Gris   
winery                                                         Rainstorm   

                                                                       3  \
country                                                               US   
description            Pineapple rind, lemon pith and orange blossom ...   
designation                                         Reserve Late Harvest   
points                                                                87   
price                                                                 13   
province                                                        Michigan   
region_1                                             Lake Michigan Shore   
region_2                                                             NaN   
taster_name                                           Alexander Peartree   
taster_twitter_handle                                                NaN   
title                  St. Julian 2013 R

<IPython.core.display.Javascript object>

In [13]:
available_data = all_data[~all_data["price"].isna()]

<IPython.core.display.Javascript object>

In [14]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


<IPython.core.display.Javascript object>

In [15]:
all_data.describe(include="all")

country                                        description designation  \
count   129908                                             129971       92506   
unique      43                                             119955       37979   
top         US  Ripe plum, game, truffle, leather and menthol ...     Reserve   
freq     54504                                                  3        2009   
mean       NaN                                                NaN         NaN   
std        NaN                                                NaN         NaN   
min        NaN                                                NaN         NaN   
25%        NaN                                                NaN         NaN   
50%        NaN                                                NaN         NaN   
75%        NaN                                                NaN         NaN   
max        NaN                                                NaN         NaN   

               points          price    province     region_1       region_2  \
count   129971.000000  120975.000000      129908       108724          50511   
unique            NaN            NaN         425         1229             17   
top               NaN            NaN  California  Napa Valley  Central Coast   
freq              NaN            NaN       36247         4480          11065   
mean        88.447138      35.363389         NaN          NaN            NaN   
std          3.039730      41.022218         NaN          NaN            NaN   
min         80.000000       4.000000         NaN          NaN            NaN   
25%         86.000000      17.000000         NaN          NaN            NaN   
50%         88.000000      25.000000         NaN          NaN            NaN   
75%         91.000000      42.000000         NaN          NaN            NaN   
max        100.000000    3300.000000         NaN          NaN            NaN   

       taster_name taster_twitter_handle  \
count       103727                 98758   
unique          19                    15   
top     Roger Voss            @vossroger   
freq         25514                 25514   
mean           NaN                   NaN   
std            NaN                   NaN   
min            NaN                   NaN   
25%            NaN                   NaN   
50%            NaN                   NaN   
75%            NaN                   NaN   
max            NaN                   NaN   

                                                    title     variety  \
count                                              129971      129970   
unique                                             118840         707   
top     Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...  Pinot Noir   
freq                                                   11       13272   
mean                                                  NaN         NaN   
std                                                   NaN         NaN   
min                                                   NaN         NaN   
25%                                                   NaN         NaN   
50%                                                   NaN         NaN   
75%                                                   NaN         NaN   
max                                                   NaN         NaN   

                    winery  
count               129971  
unique               16757  
top     Wines & Winemakers  
freq                   222  
mean                   NaN  
std                    NaN  
min                    NaN  
25%                    NaN  
50%                    NaN  
75%                    NaN  
max                    NaN

<IPython.core.display.Javascript object>

In [16]:
all_data.head(10).T

0  \
country                                                            Italy   
description            Aromas include tropical fruit, broom, brimston...   
designation                                                 Vulkà Bianco   
points                                                                87   
price                                                                NaN   
province                                               Sicily & Sardinia   
region_1                                                            Etna   
region_2                                                             NaN   
taster_name                                                Kerin O’Keefe   
taster_twitter_handle                                       @kerinokeefe   
title                                  Nicosia 2013 Vulkà Bianco  (Etna)   
variety                                                      White Blend   
winery                                                           Nicosia   

                                                                       1  \
country                                                         Portugal   
description            This is ripe and fruity, a wine that is smooth...   
designation                                                     Avidagos   
points                                                                87   
price                                                                 15   
province                                                           Douro   
region_1                                                             NaN   
region_2                                                             NaN   
taster_name                                                   Roger Voss   
taster_twitter_handle                                         @vossroger   
title                      Quinta dos Avidagos 2011 Avidagos Red (Douro)   
variety                                                   Portuguese Red   
winery                                               Quinta dos Avidagos   

                                                                       2  \
country                                                               US   
description            Tart and snappy, the flavors of lime flesh and...   
designation                                                          NaN   
points                                                                87   
price                                                                 14   
province                                                          Oregon   
region_1                                               Willamette Valley   
region_2                                               Willamette Valley   
taster_name                                                 Paul Gregutt   
taster_twitter_handle                                        @paulgwine    
title                      Rainstorm 2013 Pinot Gris (Willamette Valley)   
variety                                                       Pinot Gris   
winery                                                         Rainstorm   

                                                                       3  \
country                                                               US   
description            Pineapple rind, lemon pith and orange blossom ...   
designation                                         Reserve Late Harvest   
points                                                                87   
price                                                                 13   
province                                                        Michigan   
region_1                                             Lake Michigan Shore   
region_2                                                             NaN   
taster_name                                           Alexander Peartree   
taster_twitter_handle                                                NaN   
title                  St. Julian 2013 R

<IPython.core.display.Javascript object>

In [17]:
available_data = all_data[~all_data["price"].isna()]

<IPython.core.display.Javascript object>

In [18]:
available_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120975 entries, 1 to 129970
Data columns (total 13 columns):
country                  120916 non-null object
description              120975 non-null object
designation              86196 non-null object
points                   120975 non-null int64
price                    120975 non-null float64
province                 120916 non-null object
region_1                 101400 non-null object
region_2                 50292 non-null object
taster_name              96479 non-null object
taster_twitter_handle    91559 non-null object
title                    120975 non-null object
variety                  120974 non-null object
winery                   120975 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


<IPython.core.display.Javascript object>

In [19]:
data = available_data.fillna("0")

<IPython.core.display.Javascript object>

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120975 entries, 1 to 129970
Data columns (total 13 columns):
country                  120975 non-null object
description              120975 non-null object
designation              120975 non-null object
points                   120975 non-null int64
price                    120975 non-null float64
province                 120975 non-null object
region_1                 120975 non-null object
region_2                 120975 non-null object
taster_name              120975 non-null object
taster_twitter_handle    120975 non-null object
title                    120975 non-null object
variety                  120975 non-null object
winery                   120975 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


<IPython.core.display.Javascript object>

In [21]:
data.columns.values

array(['country', 'description', 'designation', 'points', 'price',
       'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype=object)

<IPython.core.display.Javascript object>

In [22]:
cat_cols = [
    "country",
    "designation",
    "province",
    "region_1",
    "region_2",
    "taster_name",
    "taster_twitter_handle",
    "title",
    "variety",
    "winery",
]

<IPython.core.display.Javascript object>

In [23]:
cont_cols = ["points", "price"]

<IPython.core.display.Javascript object>

In [24]:
cat_feature_cols = cat_cols

<IPython.core.display.Javascript object>

In [25]:
cont_feature_cols = ["points"]

<IPython.core.display.Javascript object>

In [26]:
feature_cols = cat_feature_cols + cont_feature_cols

<IPython.core.display.Javascript object>

In [27]:
target_col = "price"

<IPython.core.display.Javascript object>

In [28]:
def make_category_map(labels):
    return {x: i for i, x in enumerate(sorted(set(labels)))}

<IPython.core.display.Javascript object>

In [29]:
def init_category_maps(filename, df, feature_cols):
    try:
        category_maps = load_file(filename)
    except:
        category_maps = {
            feature_col: make_category_map(df[feature_col].values)
            for feature_col in feature_cols
        }
        save_file(category_maps, filename)
    return category_maps

<IPython.core.display.Javascript object>

In [30]:
category_maps = init_category_maps(TEMP_DIR / "category_maps.pickle", data, cat_cols)

<IPython.core.display.Javascript object>

In [31]:
category_maps

{'country': {'0': 0,
  'Argentina': 1,
  'Armenia': 2,
  'Australia': 3,
  'Austria': 4,
  'Bosnia and Herzegovina': 5,
  'Brazil': 6,
  'Bulgaria': 7,
  'Canada': 8,
  'Chile': 9,
  'China': 10,
  'Croatia': 11,
  'Cyprus': 12,
  'Czech Republic': 13,
  'England': 14,
  'France': 15,
  'Georgia': 16,
  'Germany': 17,
  'Greece': 18,
  'Hungary': 19,
  'India': 20,
  'Israel': 21,
  'Italy': 22,
  'Lebanon': 23,
  'Luxembourg': 24,
  'Macedonia': 25,
  'Mexico': 26,
  'Moldova': 27,
  'Morocco': 28,
  'New Zealand': 29,
  'Peru': 30,
  'Portugal': 31,
  'Romania': 32,
  'Serbia': 33,
  'Slovakia': 34,
  'Slovenia': 35,
  'South Africa': 36,
  'Spain': 37,
  'Switzerland': 38,
  'Turkey': 39,
  'US': 40,
  'Ukraine': 41,
  'Uruguay': 42},
 'designation': {'#19 Phantom Limb Belmont Vineyard': 0,
  '#50 Mon Chou': 1,
  '#SocialSecret': 2,
  '%@#$!': 3,
  '&': 4,
  "'61 Rosé": 5,
  "'A Rina": 6,
  "'Blend 105' Red Wine": 7,
  "'Na Vota": 8,
  "'S'": 9,
  "'Unfiltered'": 10,
  "'Vie!'": 11,

<IPython.core.display.Javascript object>

In [32]:
def apply_category_maps(df, category_maps):
    df = df.copy()
    for feature_col, feature_map in category_maps.items():
        df[feature_col] = df[feature_col].map(feature_map)
    return df

<IPython.core.display.Javascript object>

In [33]:
target_scaler = sk.preprocessing.RobustScaler().fit(
    data[target_col].values.reshape(-1, 1)
)

<IPython.core.display.Javascript object>

In [34]:
save_file(target_scaler, TEMP_DIR / "target_scaler.pickle")

<IPython.core.display.Javascript object>

In [35]:
feature_scaler = sk.preprocessing.RobustScaler().fit(data[cont_feature_cols])

<IPython.core.display.Javascript object>

In [36]:
save_file(feature_scaler, TEMP_DIR / "feature_scaler.pickle")

<IPython.core.display.Javascript object>

In [37]:
train_data, val_data, test_data = split_df(data, 0.4)

<IPython.core.display.Javascript object>

In [38]:
train_data.shape, val_data.shape, test_data.shape

((72585, 13), (24195, 13), (24195, 13))

<IPython.core.display.Javascript object>

In [39]:
train_data = apply_category_maps(train_data, category_maps)

<IPython.core.display.Javascript object>

In [40]:
val_data = apply_category_maps(val_data, category_maps)

<IPython.core.display.Javascript object>

In [41]:
test_data = apply_category_maps(test_data, category_maps)

<IPython.core.display.Javascript object>

In [42]:
train_data[cont_feature_cols] = feature_scaler.transform(train_data[cont_feature_cols])

<IPython.core.display.Javascript object>

In [43]:
val_data[cont_feature_cols] = feature_scaler.transform(val_data[cont_feature_cols])

<IPython.core.display.Javascript object>

In [44]:
test_data[cont_feature_cols] = feature_scaler.transform(test_data[cont_feature_cols])

<IPython.core.display.Javascript object>

In [45]:
train_data[target_col] = target_scaler.transform(train_data[[target_col]])

<IPython.core.display.Javascript object>

In [46]:
val_data[target_col] = target_scaler.transform(val_data[[target_col]])

<IPython.core.display.Javascript object>

In [47]:
test_data[target_col] = target_scaler.transform(test_data[[target_col]])

<IPython.core.display.Javascript object>

In [48]:
train_dataset = (
    tf.data.Dataset.zip(
        (
            tf.data.Dataset.from_tensor_slices(
                tuple(
                    [train_data[cal_col].values for cal_col in cat_feature_cols]
                    + [train_data[cont_feature_cols].values]
                )
            ),
            tf.data.Dataset.from_tensor_slices(train_data[target_col].values),
        )
    )
    .repeat()
    .shuffle(len(train_data))
    .batch(BATCH_SIZE)
    .prefetch(-1)
)

<IPython.core.display.Javascript object>

In [49]:
val_dataset = (
    tf.data.Dataset.zip(
        (
            tf.data.Dataset.from_tensor_slices(
                tuple(
                    [val_data[cal_col].values for cal_col in cat_feature_cols]
                    + [val_data[cont_feature_cols].values]
                )
            ),
            tf.data.Dataset.from_tensor_slices(val_data[target_col].values),
        )
    )
    .batch(BATCH_SIZE)
    .prefetch(-1)
)

<IPython.core.display.Javascript object>

In [50]:
train_data_steps = len(train_data) // BATCH_SIZE

<IPython.core.display.Javascript object>

In [51]:
def make_embedding_model(
    cat_feature_dims,
    n_cont_features,
    layer_size=256,
    dropout_rate=0.0,
    regularization=None,
    n_layers=1,
    batch_norm=True,
):

    inputs = []
    embeddings = []
    for cat_feature_dim in cat_feature_dims:
        x = keras.layers.Input(shape=(1,))
        inputs.append(x)
        x = keras.layers.Embedding(
            cat_feature_dim,
            min(50, np.sqrt(cat_feature_dim).astype(int) + 1),
            input_length=1,
            embeddings_regularizer=keras.regularizers.l1_l2(1e-2, 1e-2),
        )(x)
        x = keras.layers.Flatten()(x)
        embeddings.append(x)

    cont_inputs = keras.layers.Input(shape=(n_cont_features,))
    inputs.append(cont_inputs)
    embeddings.append(cont_inputs)

    x = keras.layers.Concatenate()(embeddings)

    for _ in range(n_layers):
        if batch_norm:
            x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dropout(dropout_rate)(x)
        x = keras.layers.Dense(
            layer_size,
            kernel_initializer=keras.initializers.he_uniform(),
            kernel_regularizer=regularization
            and keras.regularizers.l1_l2(*regularization),
        )(x)
        x = keras.layers.LeakyReLU()(x)

    x = keras.layers.Dense(1)(x)

    model = keras.Model(inputs=inputs, outputs=x)
    return model

<IPython.core.display.Javascript object>

In [52]:
embedding_model = make_embedding_model(
    [len(x) + 1 for x in category_maps.values()],
    1,
    layer_size=256,
    dropout_rate=0.5,
    regularization=(1e-8, 3e-3),
    n_layers=3,
)

<IPython.core.display.Javascript object>

In [53]:
embedding_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

<IPython.core.display.Javascript object>

In [54]:
embedding_model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.mse,
    metrics=[keras.metrics.mae],
)

<IPython.core.display.Javascript object>

In [55]:
PATIENCE = 20

embedding_model_history = embedding_model.fit(
    train_dataset,
    steps_per_epoch=train_data_steps,
    validation_data=val_dataset,
    epochs=1000,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=PATIENCE),
        tf.keras.callbacks.EarlyStopping(
            patience=PATIENCE * 2, restore_best_weights=True
        ),
    ],
    verbose=2,
)

Train for 141 steps, validate for 48 steps
Epoch 1/1000


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


141/141 - 11s - loss: 294.3701 - mean_absolute_error: 0.8564 - val_loss: 17.9692 - val_mean_absolute_error: 0.7344
Epoch 2/1000
141/141 - 8s - loss: 17.1203 - mean_absolute_error: 0.7236 - val_loss: 16.8558 - val_mean_absolute_error: 0.7435
Epoch 3/1000
141/141 - 8s - loss: 16.8844 - mean_absolute_error: 0.7037 - val_loss: 17.7347 - val_mean_absolute_error: 0.6690
Epoch 4/1000
141/141 - 8s - loss: 16.9670 - mean_absolute_error: 0.6782 - val_loss: 17.7280 - val_mean_absolute_error: 0.6345
Epoch 5/1000
141/141 - 8s - loss: 15.7704 - mean_absolute_error: 0.6091 - val_loss: 15.9441 - val_mean_absolute_error: 0.6063
Epoch 6/1000
141/141 - 8s - loss: 15.3249 - mean_absolute_error: 0.6021 - val_loss: 15.4568 - val_mean_absolute_error: 0.5652
Epoch 7/1000
141/141 - 8s - loss: 15.7225 - mean_absolute_error: 0.6093 - val_loss: 17.8598 - val_mean_absolute_error: 0.6502
Epoch 8/1000
141/141 - 8s - loss: 15.9253 - mean_absolute_error: 0.6155 - val_loss: 14.8372 - val_mean_absolute_error: 0.5442
Epo

Epoch 66/1000
141/141 - 7s - loss: 14.6456 - mean_absolute_error: 0.5771 - val_loss: 13.8771 - val_mean_absolute_error: 0.4990
Epoch 83/1000
141/141 - 7s - loss: 13.3097 - mean_absolute_error: 0.5450 - val_loss: 13.6523 - val_mean_absolute_error: 0.5313
Epoch 84/1000
141/141 - 7s - loss: 13.1425 - mean_absolute_error: 0.5332 - val_loss: 13.9889 - val_mean_absolute_error: 0.5109
Epoch 85/1000
141/141 - 7s - loss: 13.2399 - mean_absolute_error: 0.5416 - val_loss: 13.2223 - val_mean_absolute_error: 0.4998
Epoch 86/1000
141/141 - 7s - loss: 13.2889 - mean_absolute_error: 0.5394 - val_loss: 13.4914 - val_mean_absolute_error: 0.4868
Epoch 87/1000
141/141 - 7s - loss: 12.7013 - mean_absolute_error: 0.5267 - val_loss: 13.3220 - val_mean_absolute_error: 0.5101
Epoch 88/1000
141/141 - 7s - loss: 6.1412 - mean_absolute_error: 0.5062 - val_loss: 5.9488 - val_mean_absolute_error: 0.5167
Epoch 89/1000
141/141 - 7s - loss: 5.4138 - mean_absolute_error: 0.5022 - val_loss: 5.9134 - val_mean_absolute_er

141/141 - 7s - loss: 2.4093 - mean_absolute_error: 0.4195 - val_loss: 3.1252 - val_mean_absolute_error: 0.4424
Epoch 148/1000
141/141 - 7s - loss: 2.5351 - mean_absolute_error: 0.4257 - val_loss: 3.0968 - val_mean_absolute_error: 0.4368
Epoch 149/1000
141/141 - 7s - loss: 2.3147 - mean_absolute_error: 0.4149 - val_loss: 3.0821 - val_mean_absolute_error: 0.4461
Epoch 150/1000
141/141 - 7s - loss: 2.3429 - mean_absolute_error: 0.4182 - val_loss: 3.0489 - val_mean_absolute_error: 0.4402
Epoch 151/1000
141/141 - 7s - loss: 2.3338 - mean_absolute_error: 0.4178 - val_loss: 3.0559 - val_mean_absolute_error: 0.4363
Epoch 152/1000
141/141 - 7s - loss: 2.3120 - mean_absolute_error: 0.4150 - val_loss: 3.0709 - val_mean_absolute_error: 0.4337
Epoch 153/1000
141/141 - 7s - loss: 2.3181 - mean_absolute_error: 0.4159 - val_loss: 3.0420 - val_mean_absolute_error: 0.4368
Epoch 154/1000
141/141 - 7s - loss: 2.4047 - mean_absolute_error: 0.4221 - val_loss: 3.1278 - val_mean_absolute_error: 0.4405
Epoch 1

...

Epoch 344/1000
141/141 - 7s - loss: 0.5934 - mean_absolute_error: 0.3106 - val_loss: 1.4436 - val_mean_absolute_error: 0.3920
Epoch 345/1000
141/141 - 7s - loss: 0.6100 - mean_absolute_error: 0.3153 - val_loss: 1.4487 - val_mean_absolute_error: 0.3937
Epoch 346/1000
141/141 - 7s - loss: 0.5731 - mean_absolute_error: 0.3130 - val_loss: 1.4355 - val_mean_absolute_error: 0.3899
Epoch 347/1000
141/141 - 7s - loss: 0.6142 - mean_absolute_error: 0.3123 - val_loss: 1.4427 - val_mean_absolute_error: 0.3925
Epoch 348/1000
141/141 - 7s - loss: 0.5927 - mean_absolute_error: 0.3099 - val_loss: 1.4373 - val_mean_absolute_error: 0.3921
Epoch 349/1000
141/141 - 7s - loss: 0.5785 - mean_absolute_error: 0.3095 - val_loss: 1.4253 - val_mean_absolute_error: 0.3890
Epoch 350/1000
141/141 - 7s - loss: 0.6072 - mean_absolute_error: 0.3124 - val_loss: 1.4335 - val_mean_absolute_error: 0.3908
Epoch 351/1000
141/141 - 7s - loss: 0.5960 - mean_absolute_error: 0.3146 - val_loss: 1.4432 - val_mean_absolute_error:

141/141 - 7s - loss: 0.5340 - mean_absolute_error: 0.2986 - val_loss: 1.3655 - val_mean_absolute_error: 0.3852
Epoch 410/1000
141/141 - 7s - loss: 0.5527 - mean_absolute_error: 0.3011 - val_loss: 1.3842 - val_mean_absolute_error: 0.3897
Epoch 411/1000
141/141 - 7s - loss: 0.5162 - mean_absolute_error: 0.3009 - val_loss: 1.3841 - val_mean_absolute_error: 0.3901
Epoch 412/1000
141/141 - 7s - loss: 0.5262 - mean_absolute_error: 0.3014 - val_loss: 1.3854 - val_mean_absolute_error: 0.3906
Epoch 413/1000
141/141 - 7s - loss: 0.5156 - mean_absolute_error: 0.3018 - val_loss: 1.3848 - val_mean_absolute_error: 0.3905
Epoch 414/1000
141/141 - 7s - loss: 0.5393 - mean_absolute_error: 0.3028 - val_loss: 1.3778 - val_mean_absolute_error: 0.3902
Epoch 415/1000
141/141 - 7s - loss: 0.5160 - mean_absolute_error: 0.3018 - val_loss: 1.3680 - val_mean_absolute_error: 0.3889
Epoch 416/1000
141/141 - 7s - loss: 0.5511 - mean_absolute_error: 0.3044 - val_loss: 1.3761 - val_mean_absolute_error: 0.3904
Epoch 4

Epoch 475/1000
141/141 - 7s - loss: 0.4701 - mean_absolute_error: 0.3006 - val_loss: 1.3270 - val_mean_absolute_error: 0.3857
Epoch 476/1000
141/141 - 7s - loss: 0.4713 - mean_absolute_error: 0.3006 - val_loss: 1.3474 - val_mean_absolute_error: 0.3888
Epoch 477/1000
141/141 - 7s - loss: 0.5196 - mean_absolute_error: 0.3024 - val_loss: 1.3453 - val_mean_absolute_error: 0.3879
Epoch 478/1000
141/141 - 7s - loss: 0.4814 - mean_absolute_error: 0.3026 - val_loss: 1.3367 - val_mean_absolute_error: 0.3868
Epoch 479/1000
141/141 - 7s - loss: 0.4877 - mean_absolute_error: 0.3040 - val_loss: 1.3454 - val_mean_absolute_error: 0.3893
Epoch 480/1000
141/141 - 7s - loss: 0.4595 - mean_absolute_error: 0.2994 - val_loss: 1.3315 - val_mean_absolute_error: 0.3867
Epoch 481/1000
141/141 - 7s - loss: 0.4988 - mean_absolute_error: 0.3022 - val_loss: 1.3386 - val_mean_absolute_error: 0.3876
Epoch 482/1000
141/141 - 7s - loss: 0.4966 - mean_absolute_error: 0.3051 - val_loss: 1.3464 - val_mean_absolute_error:

141/141 - 7s - loss: 0.5136 - mean_absolute_error: 0.3066 - val_loss: 1.3284 - val_mean_absolute_error: 0.3884
Epoch 541/1000
141/141 - 7s - loss: 0.4713 - mean_absolute_error: 0.3075 - val_loss: 1.3463 - val_mean_absolute_error: 0.3922
Epoch 542/1000
141/141 - 7s - loss: 0.5341 - mean_absolute_error: 0.3046 - val_loss: 1.3276 - val_mean_absolute_error: 0.3889
Epoch 543/1000
141/141 - 7s - loss: 0.5202 - mean_absolute_error: 0.3050 - val_loss: 1.3296 - val_mean_absolute_error: 0.3883
Epoch 544/1000
141/141 - 7s - loss: 0.4929 - mean_absolute_error: 0.3051 - val_loss: 1.3383 - val_mean_absolute_error: 0.3905
Epoch 545/1000
141/141 - 7s - loss: 0.4930 - mean_absolute_error: 0.3079 - val_loss: 1.3267 - val_mean_absolute_error: 0.3890
Epoch 546/1000
141/141 - 7s - loss: 0.5132 - mean_absolute_error: 0.3054 - val_loss: 1.3211 - val_mean_absolute_error: 0.3882
Epoch 547/1000
141/141 - 7s - loss: 0.4543 - mean_absolute_error: 0.3055 - val_loss: 1.3277 - val_mean_absolute_error: 0.3890
Epoch 5

<IPython.core.display.Javascript object>

In [57]:
random_forest_model = sk.model_selection.RandomizedSearchCV(
    estimator=Pipeline([("rf", RandomForestRegressor(n_estimators=100))]),
    param_distributions={
        "rf__max_depth": np.arange(10, 1000),
        "rf__min_samples_leaf": np.arange(3, 20),
        "rf__max_features": ["sqrt", "log2", 0.2],
        "rf__min_samples_split": np.arange(2, 20),
        "rf__min_samples_leaf": np.arange(1, 10),
    },
    n_iter=1000,
    cv=3,
    n_jobs=-1,
    verbose=1,
)

<IPython.core.display.Javascript object>

In [58]:
random_forest_model.fit(train_data[feature_cols], train_data[target_col])

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:   42.6s
[Parallel(n_jobs=-1)]: Done 386 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 736 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1186 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1736 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 2386 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 12.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('rf',
                                              RandomForestRegressor(bootstrap=True,
                                                                    criterion='mse',
                                                                    max_depth=None,
                                                                    max_features='auto',
                                                                    max_leaf_nodes=None,
                                                                    min_impurity_decrease=0.0,
                                                                    min_impurity_split=None,
                                                                    min_samples_leaf=1,
                                                                    min_samples_split=2,
                                           

<IPython.core.display.Javascript object>

In [59]:
val_predictions = {
    "nn": target_scaler.inverse_transform(embedding_model.predict(val_dataset)),
    "rf": target_scaler.inverse_transform(
        random_forest_model.predict(val_data[feature_cols]).reshape(-1, 1)
    ),
}

<IPython.core.display.Javascript object>

In [60]:
val_targets = target_scaler.inverse_transform(val_data[[target_col]])

<IPython.core.display.Javascript object>

In [61]:
from toai.metrics import rmse

<IPython.core.display.Javascript object>

In [62]:
metrics = [rmse, sk.metrics.mean_absolute_error]

<IPython.core.display.Javascript object>

In [63]:
for metric in metrics:
    for model_type, y_pred in val_predictions.items():
        print(f"{metric.__name__} [{model_type}]: {metric(val_targets, y_pred):.4f}")

rmse [nn]: 26.2252
rmse [rf]: 27.5682
mean_absolute_error [nn]: 9.6335
mean_absolute_error [rf]: 10.3478


<IPython.core.display.Javascript object>